# OPAN5505 Lab 3 Assignment

## Background

You've been engaged by the Federal Aviation Administration to perform an analysis on "strike" data. A "strike" refers to when an aircraft and an object within the air (typically wildlife) collide.

## Prepare your environment

You will need the `polars` package for this assignment. We'll also use `datetime` for date handling.

In [2]:
# Import required libraries
import polars as pl
import datetime

## Load your dataset

Read the `faa_strikes.txt` dataset into Polars. Name the resulting DataFrame `strikes`.

In [5]:
from google.colab import files
uploaded = files.upload()

Saving faa_strikes.txt to faa_strikes.txt


## Question 1: Running total of strikes by day through 2013

You are interested in seeing the running total of strikes over time (`Collision Date and Time`) on a daily basis. Using the `strikes` dataframe, create a new column that removes the time information but keeps the date information from the `Collision Date and Time` column (call this new field `date`). Next, aggregate the `Number of Strikes` by day (call this new field `daily_strikes`). Sort the data in ascending order by `date`. Then, create the running total of `daily_strikes` (name the new field `strikes_cumulative`) and filter to only the records **up to and including** the `date` of 2013-12-31. Name the resulting data frame `running_total_strikes`.

*Please place a Python code block and code below. The output should be a DataFrame. You must use a window function to receive credit on this question.*

In [13]:
# Your code here
import polars as pl

strikes = pl.read_csv("faa_strikes.txt", separator="\t", null_values=["NA", ""])

strikes = strikes.with_columns(pl.col("Collision Date and Time").str.strptime(pl.Datetime, strict=False).dt.date().alias("date"))

daily_strikes = (strikes.group_by("date").agg(pl.col("Number of Strikes").sum().alias("daily_strikes")).sort("date"))

running_total_strikes = (daily_strikes.with_columns(pl.col("daily_strikes").cum_sum().alias("strikes_cumulative")).filter(pl.col("date") <= pl.date(2013, 12, 31)))
print(running_total_strikes.head())
print(running_total_strikes.shape)

shape: (5, 3)
┌────────────┬───────────────┬────────────────────┐
│ date       ┆ daily_strikes ┆ strikes_cumulative │
│ ---        ┆ ---           ┆ ---                │
│ date       ┆ i64           ┆ i64                │
╞════════════╪═══════════════╪════════════════════╡
│ 2000-01-02 ┆ 1             ┆ 1                  │
│ 2000-01-03 ┆ 2             ┆ 3                  │
│ 2000-01-05 ┆ 3             ┆ 6                  │
│ 2000-01-06 ┆ 1             ┆ 7                  │
│ 2000-01-08 ┆ 4             ┆ 11                 │
└────────────┴───────────────┴────────────────────┘
(4776, 3)


## Question 2: States with the third highest financial cost

The FAA is interested in the financial cost of strikes for each `Origin State`. Using the `strikes` dataframe: first, sum `Cost: Total $` by `Origin State` (name the resulting column `damage`) then `rank` these states in order of which state had the most financial `damage` (name the new column with rank information, `ranking`). Next, filter to show the row with the third highest `ranking`. Use a window function to answer this question. Name the resulting dataframe `damage_state`.

*Please place a Python code block and code below. The output should be a DataFrame. You must use a window function to receive credit on this question.*

In [15]:
# Your code here
damage_state = (strikes.group_by("Origin State").agg(pl.col("Cost: Total $").sum().alias("damage")).with_columns(pl.col("damage").rank("dense", descending=True).alias("ranking")).filter(pl.col("ranking") == 3))

print(damage_state)

shape: (1, 3)
┌──────────────┬──────────┬─────────┐
│ Origin State ┆ damage   ┆ ranking │
│ ---          ┆ ---      ┆ ---     │
│ str          ┆ i64      ┆ u32     │
╞══════════════╪══════════╪═════════╡
│ California   ┆ 29671432 ┆ 3       │
└──────────────┴──────────┴─────────┘


## Question 3: What are the second costliest Species Groups for each Aircraft Type?

The FAA wants to know if some species groups are more dangerous to particular types of aircraft. Using the `strikes` dataframe: first, sum financial damage (`Cost: Total $`) information by `Aircraft: Type` and `Wildlife: Species Group` (name the new field `damage`). Rank the rows within each `Aircraft: Type` based on the greatest amount of `damage`; when performing the ranking function, name the new column `ranking`. Return the rows that represent the `Wildlife: Species Group` values that caused the **second most** financial `damage` to each `Aircraft: Type`. Name the resulting dataframe `type_species`.

*Please place a Python code block and code below. The output should be a DataFrame. You must use a ranking function to receive credit on this question.*

In [17]:
# Your code here
type_species = (strikes.group_by(["Aircraft: Type", "Wildlife: Species Group"]).agg(pl.col("Cost: Total $").sum().alias("damage")).with_columns(pl.col("damage").rank("dense", descending=True).over("Aircraft: Type").alias("ranking")).filter(pl.col("ranking") == 2))

print(type_species)

shape: (3, 4)
┌────────────────┬─────────────────────────┬──────────┬─────────┐
│ Aircraft: Type ┆ Wildlife: Species Group ┆ damage   ┆ ranking │
│ ---            ┆ ---                     ┆ ---      ┆ ---     │
│ str            ┆ str                     ┆ i64      ┆ u32     │
╞════════════════╪═════════════════════════╪══════════╪═════════╡
│ null           ┆ Vultures                ┆ 56227    ┆ 2       │
│ Airplane       ┆ Kites, Hawks, Eagles    ┆ 48989287 ┆ 2       │
│ Helicopter     ┆ Vultures                ┆ 116748   ┆ 2       │
└────────────────┴─────────────────────────┴──────────┴─────────┘


## Question 4: Which days had the greatest positive jump in strikes?

The FAA wants to investigate which days had the largest increase in strikes from the previous day. Using the `strikes` dataframe: sum the `Number of Strikes` measure by day (note: not by day and time). You can use a similar technique from Question 1 to create a new column from `Collision Date and Time` without time information and with date information (name this column `date`). Next, compute the previous day's strikes using a window function (name the new column `previous_day`), and calculate the difference of strikes between the current day and the previous day (name the new column `delta_strikes`). Sort the resulting data by `delta_strikes` in descending order to find which days had the highest increase of strikes from the previous day. Name the resulting dataframe `greatest_strike_increase`.

**Hint:** This code is more involved that other exercises, so I will explicitly name the steps to answer the question.

1. Start with the `strikes` data frame
2. Create a new column called `date` which removes the time information in the `Collision Date and Time` column
3. Sum the `Number of Strikes` by `date`, and name the new column `daily_strikes`
4. Sort by the `date` column in ascending order
5. Create a new column called `previous_day` to calculate what the `daily_strikes` were in the previous day
6. Create a new column called `delta_strikes` which subtracts `previous_day` from `daily_strikes`
7. Sort in descending order by `delta_strikes`

*Please place a Python code block and code below. The output should be a DataFrame. You must use a window function to receive credit on this question.*

In [33]:
# Your code here
import polars as pl

strikes_w_date = strikes.with_columns(pl.col("Collision Date and Time").str.strptime(pl.Datetime, strict=False).dt.date().alias("date"))


daily_strikes = (strikes_w_date.group_by("date").agg(pl.col("Number of Strikes").sum().alias("daily_strikes")).sort("date"))


greatest_strike_increase = (daily_strikes.with_columns([pl.col("daily_strikes").shift().alias("previous_day"),(pl.col("daily_strikes") - pl.col("daily_strikes").shift()).alias("delta_strikes")]).sort("delta_strikes", descending=True))

print(greatest_strike_increase.head(10))

shape: (10, 4)
┌────────────┬───────────────┬──────────────┬───────────────┐
│ date       ┆ daily_strikes ┆ previous_day ┆ delta_strikes │
│ ---        ┆ ---           ┆ ---          ┆ ---           │
│ date       ┆ i64           ┆ i64          ┆ i64           │
╞════════════╪═══════════════╪══════════════╪═══════════════╡
│ 2000-01-02 ┆ 1             ┆ null         ┆ null          │
│ 2010-10-29 ┆ 25            ┆ 7            ┆ 18            │
│ 2014-08-25 ┆ 25            ┆ 8            ┆ 17            │
│ 2012-06-28 ┆ 21            ┆ 5            ┆ 16            │
│ 2009-07-07 ┆ 21            ┆ 7            ┆ 14            │
│ 2013-11-04 ┆ 22            ┆ 8            ┆ 14            │
│ 2014-10-28 ┆ 24            ┆ 10           ┆ 14            │
│ 2009-09-13 ┆ 16            ┆ 3            ┆ 13            │
│ 2009-10-05 ┆ 18            ┆ 5            ┆ 13            │
│ 2011-07-29 ┆ 23            ┆ 10           ┆ 13            │
└────────────┴───────────────┴──────────────┴──────────

## Question 5: Which single day had the greatest increase in strikes for each `Aircraft: Type`?

FAA was interested in the exercise from the last question but now wants to determine the largest delta between days for each `Aircraft: Type`. What are the days that had largest positive change in strikes for each `Aircraft: Type`? In order to answer this question, you will need to perform the exercise from Question 4, but this time include a grouping by `Aircraft: Type`. After grouping by `Aircraft: Type`, add a column called `ranking` and use a window function to determine the day with the largest increase in strikes from the previous day. Name the resulting data frame `greatest_strike_increase_type`

NOTE: Helicopters are not struck with a high frequency and will not show up in your analysis; this is fine.

In [42]:
# Your code here
import polars as pl

strikes_w_date = strikes.with_columns(pl.col("Collision Date and Time").str.strptime(pl.Datetime, strict=False).dt.date().alias("date"))

daily_by_type = (strikes_w_date.group_by(["Aircraft: Type", "date"]).agg(pl.col("Number of Strikes").sum().alias("daily_strikes")).sort(["Aircraft: Type", "date"]))

greatest_strike_increase_type = (daily_by_type.with_columns([pl.col("daily_strikes").shift().over("Aircraft: Type").alias("previous_day"),(pl.col("daily_strikes") - pl.col("daily_strikes").shift().over("Aircraft: Type")).alias("delta_strikes"),]).with_columns(pl.col("delta_strikes").rank("dense", descending=True).over("Aircraft: Type").alias("ranking")).filter((pl.col("ranking") == 1) & (pl.col("delta_strikes") > 0)))

print(greatest_strike_increase_type)

shape: (6, 6)
┌────────────────┬────────────┬───────────────┬──────────────┬───────────────┬─────────┐
│ Aircraft: Type ┆ date       ┆ daily_strikes ┆ previous_day ┆ delta_strikes ┆ ranking │
│ ---            ┆ ---        ┆ ---           ┆ ---          ┆ ---           ┆ ---     │
│ str            ┆ date       ┆ i64           ┆ i64          ┆ i64           ┆ u32     │
╞════════════════╪════════════╪═══════════════╪══════════════╪═══════════════╪═════════╡
│ null           ┆ 2014-08-04 ┆ 5             ┆ 1            ┆ 4             ┆ 1       │
│ Airplane       ┆ 2014-08-25 ┆ 25            ┆ 8            ┆ 17            ┆ 1       │
│ Helicopter     ┆ 2010-07-21 ┆ 2             ┆ 1            ┆ 1             ┆ 1       │
│ Helicopter     ┆ 2012-11-02 ┆ 2             ┆ 1            ┆ 1             ┆ 1       │
│ Helicopter     ┆ 2013-10-28 ┆ 2             ┆ 1            ┆ 1             ┆ 1       │
│ Helicopter     ┆ 2014-08-14 ┆ 2             ┆ 1            ┆ 1             ┆ 1       │
└──────